In [ ]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, select, ForeignKey, alias

# 创建数据库引擎和元数据对象
engine = create_engine('sqlite:///:memory:')
metadata = MetaData()

# 创建 employee 表
employee_table = Table(
    'employee', metadata,
    Column('id', Integer, primary_key=True),
    Column('manager_id', None, ForeignKey('employee.id')),
    Column('name', String(255))
)
# 创建表
metadata.create_all(engine)

# 插入一些示例数据
with engine.connect() as conn:
    conn.execute(employee_table.insert(), [
        {'name': 'Fred', 'manager_id': None},
        {'name': 'Bob', 'manager_id': 1},
        {'name': 'Charlie', 'manager_id': 2},
        {'name': 'David', 'manager_id': 1},
    ])

# 使用别名查询由名为 'Fred' 的员工管理的所有员工
with engine.connect() as conn:
    # 创建别名
    employee_alias = alias(employee_table)

    # 创建查询
    stmt = (
        select([employee_table.c.name])
        .select_from(employee_table.join(employee_alias, employee_table.c.manager_id == employee_alias.c.id))
        .where(employee_alias.c.name == 'Fred')
    )

    # 执行查询
    result = conn.execute(stmt)

    # 打印结果
    for row in result:
        print(f"Employee managed by Fred: {row.name}")
# Employee managed by Fred: Bob
# Employee managed by Fred: David
